# Goals

## Business Goals
For our project we determined three overarching business goals that we would focus on, which we will then expand upon in the analytics goals and general model. Firstly, our central business goal and the overarching goal of the project is to **maximize yearly profit** (via optimization) for a specific listing on Airbnb. All other goals are aimed either at modifying this central goal or constraining it to make our final solution more viable.
Our second business goal is to **create a dynamic pricing tool** that determines a daily price competitive with other localized Airbnb listings. By determining price at the scale of the smallest possible rental period (daily) we wanted to create a model that was as flexible as possible. This means that our optimization model could be specified down to the specific days that a user would/would not expect to be renting their listing, hopefully further increasing the validity of our end result.
Our third main business goal is to **create an easily repeatable design process** for expansion to other locations. Because our current model has a narrow scope in terms of location, we wanted to be certain that as we constructed our model, our design and the choices we made could be easily replicated in the future so that this model could be adapted to have a more universal utility.

## Analytics Goals
Once we had established our business goals, and what we wanted to accomplish in this project, we then created 3 main analytics goals in order to determine how we wanted to meet our goals. Our first analytics goal is to find the **function of demand** in terms of price via k-means clustering. In building our model, we needed a way to effectively project demand given a set of predetermined parameters; therefore, we chose to use k-means clustering on listings from identical neighborhoods with similar attributes. This ensured that the clusters were from properties as similar as possible, so that the differences in attributes were the only things affecting rentals and we could determine their individual influences on our demand variable.
Our second analytics goal is to **predict the intrinsic value** of a property by running a kNN regression using attributes of that property. Similar to the previous goal, we wanted to ensure our  valuation of properties was as accurate as possible, so by using kNN on the many variables presented in the Airbnb dataset (number of beds, amenities, etc.), we knew that the classes we were creating within our regression ensured that any factors not included in our model would have a minimal effect, as they would be adequately described by the NN algorithm.
Our third analytics goal is to calculate the optimal daily price through an optimization model of yearly profit. This goal describes our main business goal in more concrete analytical terms. Once we knew we would be using an optimization model, we could structure our analysis around the component pieces necessary; a modeled demand variable, intrinsic amenities’ additive values for properties, and valuation of costs vs. revenues. By framing our problem as a typical revenue - cost optimization, we could outline our general structure and begin to create the pieces necessary for our final model.

# System Architecture
<img width="720" alt="screenshot 2018-11-13 00 06 26" src="https://user-images.githubusercontent.com/30711638/48392141-fce4a480-e6d7-11e8-92bf-1c542a421edd.png">

Since the focus of our project is to determine the most optimal listing price, on a daily basis level, to maximize the long-term profit, we initially struggled to identify the better approach between two common solution methods, which are time series regression and optimization modeling. Time series regression is ideal for dealing with the daily fluctuation of price, and it can take into account the effect of other variables such as property types, amenities, the safety of Airbnb on the listing prices. However, time series regression doesn’t guarantee that the “best fitted” price is the “optimal” price that will maximize our profit. On the other hand, optimization is ideal for finding the “optimal” prices, but if the optimization model will be very complex if we try to find optimal price for 365 days, as well as include the effect of Airbnb characteristics on the daily prices. After a thorough consideration, we found a way to combine the regression model into the optimization by using it in our variable creation process; in addition we applied several assumptions to simplify the model while still taking the time series factors into consideration.  

The system architecture (figure 1) shows how the model will be implemented both with data sources and methodologies. Mentioned in Solution Strategy I, we separated the datasets from two sources, being the neighborhood GeoJSON file, which extracts the accurate locations of all the listings in San Francisco, and the historical listing information that includes the facility details and booking statistics of each listing. With the GeoJSON file, we divided the listings into nearby neighborhoods, and then we used clustering analysis within each neighborhood to group the listings with similar features without having to account for locational variations in property valuation. The results gave us a list of clusters which we used to identify the primary listings that will potentially compete with each other. Combining Airbnb historical listing information and the clustering results, we then went through multiple iterations of regression analysis for each cluster before incorporating them into a singular demand function that varied in terms of listing price.  Knowing that each listing must have an inherent, baseline price that would prevent any negative valuation, we imported the historical data to kNN regression and determined a baseline value for each listing. Finally, we incorporated both the demand function and the baseline price into our optimization model to generate the best price, given the booking probability on any day by our demand variable.


# Data Model
## Occupancy Rate 
As found in the Overview of the Airbnb Community in San Francisco published by Airbnb, the average length of stay per guest is 4.2 nights. We assumed each listing has 4.2 days as an average lengths of stay per booking. Since we were not able to find a clear number for the ratio  of guests making a booking who leave a review for Airbnb, we assumed the review rate to be equal to 0.5, which will be used as a constant throughout the estimation. To prevent artificially high results, we also assumed the maximum occupancy rate cannot exceed 0.95, meaning even the busiest of listings will have several nights a month in which they go unrented. With these assumptions and constants, we generated the formulation of estimated occupancy rate shown below:
<img width="658" alt="screenshot 2018-11-13 03 53 15" src="https://user-images.githubusercontent.com/30711638/48401720-c3bc2c80-e6f7-11e8-84e7-82cd1dbe25dd.png">

## Demand Function
Initially, we planned to have two separate models for predicting the listing price and the demand. One such model used k-Means to cluster all Airbnb listings within 3 miles radius to find groups of nearby similar properties, and took the average occupancy rate to estimate the expected monthly demand within each cluster. We then had another independent k Nearest Neighbor regression model to forecast daily listing price based on listing attributes that best represent the property values (amenities, safety, etc). Finally, we estimated the expected monthly profit by taking the daily average listing price of each month and multiplied by the predicted monthly demand ( the predicted number of days that a property is booked in a month).  
However, after many attempts to improve this model’s accuracy, we found that it was unreasonable to predict demand and listing price separately, primarily because these two quantities are heavily correlated with each other. For example, the demand will likely decrease if the listing price increases, since customers can be expected to make smart decisions and switch their booking to a cheaper Airbnb with similar characteristics. Therefore, we decided to model demand first, and then incorporate it into our price function. This meant that we needed to construct a demand function that captures competitor factors, listing characteristics, and time fluctuations.

<img width="659" alt="screenshot 2018-11-13 03 49 02" src="https://user-images.githubusercontent.com/30711638/48401408-1a753680-e6f7-11e8-9a04-f78caca21b7e.png">


## Competition Analysis
We model the competitor factor by using the same approach that we initially planned to apply for our monthly demand estimation. Customers tends to choose their Airbnb with a specific location in mind, so all listings that are located close together (within 3-mile radius) will be more likely to compete with each other. Going beyond this, the characteristics and quality of the listing should have an almost equal, and occasionally greater impact on determining occupancy rates relative to those properties competing with one another. From these guides we elected to use k-Means to cluster similar properties within a 3-mile radius distance (figure 2&3). 
<img width="661" alt="screenshot 2018-11-13 01 50 13" src="https://user-images.githubusercontent.com/30711638/48395835-7aafac80-e6e6-11e8-97ee-334b0695feb4.png">

## Formulate Demand Function of “Competition” Group

For each cluster, we have a set of data with the X-variable being the listing price and the Y-variable being the demand represented by the occupancy rate. From this we will fit either a  linear or polynomial regression model onto this dataset to find the best-fitted function of demand  
$D_{xij} = \alpha* x_{ij}^2 + *x_{ij} + \beta$.  
We then put this demand function into the optimization model. The objective is to maximize profit in one year, so the formula to calculate profit for each day is the listing price on that day multiplied by the demand function on that day, represented as $D_{xij} * x_{xij}$. It is worth noting that at this point in the analysis, demand in our dataset is the occupancy rate, which represents the probability a property is booked on a given day. We will mention the details of how to calculate the optimal yearly profit in the Decision Model later. 
	After several trials, we determined that the relationship between demand and price is not always inversely correlated together and straightforward linear; outlier-type cases in particular don’t fit into our model, meaning that extremely expensive listings seem to have different rules of demand governing them, which is reasonable but not accounted for in our limited model. 
<img width="555" alt="screenshot 2018-11-13 01 55 13" src="https://user-images.githubusercontent.com/30711638/48396056-2eb13780-e6e7-11e8-8c02-1a989dca7e46.png">

## Prediction of Intrinsic Property Values
We first set out to incorporate the property-specific characteristics into the model to customize the pricing model for each listing; as mentioned previously if we include too many variables into an optimization, we are less likely to have a valid solution. Instead we used our regression model to determine a “baseline” price, which we define as a price mark that indicates the intrinsic value of an Airbnb property without considering time series fluctuation (seasonalities, weekend vs weekday). Then in the optimization model we display the daily price $x_{ij}$  in terms of the defined baseline price with the constraint  
$x_{ij} = \beta_{ij} * x_0$  
The optimization model will find the optimal coefficient ij, and consequently the optimal daily listing price xij. 
	Additionally we used the k Nearest Neighbor (kNN) regression to determine the most accurate fitted baseline price for each listing, based on attributes that we decided will determine the value of a listing such as location, amenities, review scores, safety, etc. The reason we selected the kNN algorithm is because it determines the response variable Y based on the values of X-variables from k neighbors, which in our case is the values of nearby Airbnb competitors. This is a good model fit for our data because in real life, it is usually the case that real estate, hotel, and Airbnb values are heavily influenced by their surrounding competitors as a function of their location.


# Optimization Model
### Time series factors
In attempting to find the daily optimal price, we determined that our optimization model will have up to 365 price variables x1,..., x365. We originally simplified the model by assuming that price is relatively consistent throughout one month. However when further analyzing past data, we observed that Airbnb’s demand is generally higher during weekend than weekday, so we further delineated our model into two separate optimization models for both weekends and weekdays. From this we arrived at our model’s assumptions that listing price should be constant for all weekends within a month and for all weekdays within a month. 

### Decision Variables:
- ij: binary variable which is 1 if customers decide to book Airbnb property on day j of month i, equal to 0 otherwise
- $x_{ij}$: price of Airbnb property on day j of month i
Parameters:
- $C_V$ is variable cost that includes cleaning fee, utilities, guest-included fee, deterioration fee on amenities. For simplicity of our model, we will let users independently input $C_V$ to the application, allowing us to treat $C_V$ as a constant. Moreover, we assume $C_V$ will be incurred only on the days where bookings happened.  
- $I_O$ is initial investment the property owner supplies, which potentially includes the real estate or reimbursement costs, amenity purchases, maintenance cost or any other upfront cost. For simplicity of our model, we will let users input the estimate initial investment on their property, allowing us to treat this as a fixed cost throughout our model.  
- $D_{ij}$: demand of day j on month i, represented by the probability that the listing is actually booked on a specific day, given the listing price xij. $D_{ij}$ is the function of $x_{ij}$ and is determined by performing regression analysis on the cluster that the specific Airbnb property belongs to.

### Optimization Model

At this point we had constructed the components necessary for us to begin formulating our final optimization model. As defined above, our demand function will indicate the probability that the property is booked on day i, month j; if this “booked” probability is bigger than 50%, then we assume the property is marked as booked in our model on day i month j. This then allows the host to collect the optimal listing price, $x_(ij)$ and subtract the variable cost, $C_V$. The binary variable ij is treated as a control because the bookings on the property as a function of its probability. Therefore, the function to estimate monthly profit (months simplified to all be 30 days in length) will be:

<img width="130" alt="screenshot 2018-11-13 00 11 57" src="https://user-images.githubusercontent.com/30711638/48392310-c22f3c00-e6d8-11e8-9ec6-870bb1972dac.png">
 If occupancy of that month is 45%, then the expected number of days booked in that month is 45% * 30 days = 14 days. We can then retrieve the exact date of that month where j = 1 or D(xj) > 50%, and sum up their corresponding predicted prices to get the expected profit of that month. 

### Objective function:
The objective function is to maximize yearly profit, which equals to the revenue subtracted from the cost. The revenue is the sum of optimal listings prices on the days that bookings happen, or in the words, where ij = 1. We take the summation of both weekend and weekday (determined by index j) and of 12 months (determined by index i).

<img width="655" alt="screenshot 2018-11-13 00 13 17" src="https://user-images.githubusercontent.com/30711638/48392350-f0148080-e6d8-11e8-8853-b7197ddeed35.png">

In [ ]:
##ISYE 4803 
##Airbnb Project 
from __future__ import division
from pyomo.environ import *
import pyomo.opt
import math
import pprint 
import csv



#####Please substitute the input with the real input data
##########Parameters##########
Cv = 20.0
I0 = 300.0
X_0=100.0


##########Decision_Variables###########
# i = 1.......12 
# j = 1, 2, 3, 4/ j = 5,6,7
# binary variable(if customer decide to book airbnb property)


numDays = 6
model = ConcreteModel()

month = [1,2,3,4,5,6,7,8,9,10,11,12]

model.month = Set(initialize = month)
model.day = RangeSet(1,7)

model.alpha_ij = Var(model.month, model.day, within=Binary)
model.x_ij = Var(model.month,model.day, within=PositiveReals)
model.d_ij = Var(model.month,model.day, within=PositiveReals)

model.boundX = ConstraintList()
for m in model.month:
    for d in model.day:
        model.boundX.add(model.x_ij[m,d] == model.alpha_ij[m,d] * X_0)
        # model.boundX.add(model.x_ij[m,d] = model.alpha_ij[m,d]*X_0)
        model.boundX.add(model.d_ij[m,d] == -0.2417 * model.x_ij[m,d]*model.x_ij[m,d] +18.039*model.x_ij[m,d]+551.46)
model.obj = Objective(expr = sum(model.alpha_ij[m,d] * model.x_ij[m,d]
-model.alpha_ij[m,d]*Cv-I0 for m in model.month for d in model.day), sense = maximize)

opt = SolverFactory('gurobi')
instance = model.create()
results = opt.solve(instance)
instance.display()